In [19]:
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html

app = JupyterDash(__name__)

app.layout = html.Div(children = [
    html.H1("Demo: Plotly Express and Dash"),
    html.H2("This is a sub-header, yes I am")
    
])

app.run_server(mode="jupyterlab", port=8086)

This is off to a rough start but I don't think it's due to Dash. The demo shows JupyterDash brings in a live updating window. I am still having to open a new tab, put in the localhost and reload after all the changes. And that's after getting past a wall of error messages. But it's marginally good enough to continue.

In [20]:
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

app = JupyterDash(__name__)

app.layout = html.Div(children = [
    html.H1("Demo: Plotly Express and Dash"),
    html.H2(id="the_output"),
    dcc.Input(id="the_input")
    
])

@app.callback(Output('the_output', 'children'),
             [Input('the_input', 'value')])

def cd(input_value):
    return input_value


app.run_server(mode="jupyterlab", port=8086)

Still working. 

In [30]:
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

df = px.data.gapminder()
col_options = [dict(label=x, value=x) for x in df['year'].unique()]


app = JupyterDash(__name__)

app.layout = html.Div(children = [
    html.H1("Demo: Plotly Express and Dash"),
    dcc.Dropdown(id='year', value=2007, options=col_options),
    dcc.Graph(id="graph", figure={})
    
])

@app.callback(Output('graph', 'figure'),
             [Input('year', 'value')])

def cd(year):
    year = year if year else 2007
    df_year = df.query('year == @year')
    return px.scatter(df_year, x='gdpPercap', y='lifeExp', size='pop', size_max=60, height=600, color='continent')


app.run_server(mode="jupyterlab", port=8086)

OK, we had this before with plotly express and a lot less code.

In [45]:
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

df = px.data.gapminder()
col_options = [dict(label=x, value=x) for x in df['year'].unique()]


app = JupyterDash(__name__)

app.layout = html.Div(children = [
    html.H1("Demo: Plotly Express and Dash"),
    dcc.Dropdown(id='year', value=2007, options=col_options),
    dcc.Graph(id="graph", figure={}),
    dcc.Graph(id="graph2", figure={})
    
])


@app.callback(Output('graph', 'figure'), [Input('year', 'value')])

def cb(year):
    year = year if year else 2007
    df_year = df.query('year == @year')
    return px.scatter(df_year, x='gdpPercap', y='lifeExp', size='pop', size_max=60, height=400, color='continent', 
                     custom_data=[df_year.index]).update_layout(dragmode='lasso')


                   
@app.callback(Output('graph2', 'figure'),
             [Input('year', 'value'), Input('graph', 'selectedData')])

def display_selected_data(year, selectedData):
    year = year if year else 2007
    df_year = df.query('year == @year')
    
    if selectedData:
        indices = [point['customdata'][0]
                   for point in selectedData['points']]
        df_year = df_year.loc[indices]
    
    return px.scatter_geo(df_year, locations='iso_alpha', size='pop', height=600)

app.run_server(mode="jupyterlab", port=8086)